Input reviewer name as argument in get_reviewer_data(['Dennis+Schwartz', 'James+Berardinelli']) method as list.
Method searches for file in scale_whole_review/Dennis+Schwartz/txt.parag and scaledata/James+Berardinelli directories. And then, execute all cells with Shift+Enter

In [122]:
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

import re
import numpy as np
import transformers as transform
import torch

In [123]:
from sklearn.linear_model import LogisticRegression

In [124]:
class Bert_Regression:

    model_class = transform.BertModel
    tokenizer_class = transform.BertTokenizer
    pretrained_weights = 'bert-base-uncased'
    tokenizer = 0
    model = 0

    def initilize_tokenizer(self):
        self.tokenizer = self.tokenizer_class.from_pretrained(self.pretrained_weights)
        return self.tokenizer

    def initilize_model(self):
        self.model = self.model_class.from_pretrained(self.pretrained_weights)
        return self.model

    def clear_model_and_tokenizer(self):
        self.tokenizer = None
        self.model = None

    def get_reviewer_data(self, reviewer_names):

        reviewer_ids = pd.DataFrame(columns=['ids'])
        reviewer_ratings = pd.DataFrame(columns=['ratings'])

        reviewer_file_ids = ['scaledata/'+reviewer_name+'/id.'+reviewer_name for reviewer_name in reviewer_names]
        reviewer_file_ratings = ['scaledata/'+reviewer_name+'/rating.'+reviewer_name for reviewer_name in reviewer_names]
        reviews_folder = ['scale_whole_review/'+reviewer_name+'/txt.parag' for reviewer_name in reviewer_names]

        count = 0
        
        for reviewer_id in reviewer_file_ids:
            
            id_rating = pd.read_csv(reviewer_id, names = ['ids'])
            id_rating['full_review'] = ''
            
            for id in id_rating['ids']:
                path = reviews_folder[count]+'/'+str(id)+'.txt'
                f = open(path, 'r', encoding='cp1252')
                text = f.read()
                text = re.sub(r'[^\w]', ' ', text)
                text = re.sub("\d+", "", text)
                text = text.lower()
                id_rating.loc[id_rating['ids'] == id, 'full_review'] = text
                # id_rating.loc[id == dennis_fulls, 'full_review'] = text
                # print(id_rating)
                f.close()
                # print(path)
                
            reviewer_ids = pd.concat([reviewer_ids, id_rating], axis=0)
            count = count + 1
        
        for reviewer_rating in reviewer_file_ratings:
            # print(reviewer_id)
            id_rating = pd.read_csv(reviewer_rating, names = ['ratings'])#.round()
            
            reviewer_ratings = pd.concat([reviewer_ratings, id_rating], axis=0)
            # print(reviewer_ratings)
            

        reviewer_ids.reset_index(drop=True, inplace=True)
        reviewer_ratings.reset_index(drop=True, inplace=True)

        reviewer_data = pd.concat([reviewer_ids, reviewer_ratings], axis=1)


        return reviewer_data
    
    def align_data(self, tokenized):
        max_length = max([len(i) for i in tokenized])
        
        features = pd.DataFrame(columns=range(max_length))
        
        for i in range(len(tokenized)):
            # features = features.append(pd.DataFrame(dennis_full['tokenized_sentences'][i]).transpose())
            features = pd.concat([features, pd.DataFrame(tokenized[i]).transpose()])
            # features = features.shift(-1)
            # features.iloc[-1, :] = features.append(pd.DataFrame(dennis_full['tokenized_sentences'][i]).transpose())
        features.reset_index(drop=True, inplace=True)

        return features
                

In [125]:
x = Bert_Regression()

In [126]:
reviewers_data = x.get_reviewer_data(['James+Berardinelli'])

In [127]:
reviewers_data

,ids,full_review,ratings
0,2321,i didn t expect ghost in the machine to be thi...,0.05
1,3337,admittedly with a title like the mangler you...,0.05
2,6511,cast christopher lambert natasha henstridge ...,0.05
3,6912,cast john leguizamo jeffrey jones edoardo b...,0.05
4,2240,starring jim varney ron k james duke ernsb...,0.09
...,...,...,...
1302,3708,poland shown at the philadelphia festival of...,0.96
1303,4359,united states u s availability ny la ...,0.96
1304,2354,date released ny la elsewhere run...,0.97
1305,2757,i don t usually do reviews of older movies bu...,0.97


In [128]:
tokenizer = x.initilize_tokenizer()

In [129]:
reviewers_data['tokenized_data'] = reviewers_data['full_review'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, padding='max_length', max_length=3210)))

In [130]:
reviewers_data['tokenized_data']

0       [101, 1045, 2134, 1056, 5987, 5745, 1999, 1996...
1       [101, 4914, 2135, 2007, 1037, 2516, 2066, 1996...
2       [101, 3459, 5696, 12838, 17399, 21863, 3367, 9...
3       [101, 3459, 2198, 4190, 10179, 20722, 2080, 10...
4       [101, 4626, 3958, 13075, 5420, 6902, 1047, 250...
                              ...                        
1302    [101, 3735, 3491, 2012, 1996, 4407, 2782, 1997...
1303    [101, 2142, 2163, 1057, 1055, 11343, 6396, 247...
1304    [101, 3058, 2207, 6396, 2474, 6974, 2770, 3091...
1305    [101, 1045, 2123, 1056, 2788, 2079, 4391, 1997...
1306    [101, 3058, 2207, 6396, 2474, 6974, 2770, 3091...
Name: tokenized_data, Length: 1307, dtype: object

In [131]:
reviewers_data

,ids,full_review,ratings,tokenized_data
0,2321,i didn t expect ghost in the machine to be thi...,0.05,"[101, 1045, 2134, 1056, 5987, 5745, 1999, 1996..."
1,3337,admittedly with a title like the mangler you...,0.05,"[101, 4914, 2135, 2007, 1037, 2516, 2066, 1996..."
2,6511,cast christopher lambert natasha henstridge ...,0.05,"[101, 3459, 5696, 12838, 17399, 21863, 3367, 9..."
3,6912,cast john leguizamo jeffrey jones edoardo b...,0.05,"[101, 3459, 2198, 4190, 10179, 20722, 2080, 10..."
4,2240,starring jim varney ron k james duke ernsb...,0.09,"[101, 4626, 3958, 13075, 5420, 6902, 1047, 250..."
...,...,...,...,...
1302,3708,poland shown at the philadelphia festival of...,0.96,"[101, 3735, 3491, 2012, 1996, 4407, 2782, 1997..."
1303,4359,united states u s availability ny la ...,0.96,"[101, 2142, 2163, 1057, 1055, 11343, 6396, 247..."
1304,2354,date released ny la elsewhere run...,0.97,"[101, 3058, 2207, 6396, 2474, 6974, 2770, 3091..."
1305,2757,i don t usually do reviews of older movies bu...,0.97,"[101, 1045, 2123, 1056, 2788, 2079, 4391, 1997..."


In [132]:
aligned_data_for_ML = x.align_data(reviewers_data['tokenized_data'])
aligned_data_for_ML

,0,1,2,3,4,5,6,7,8,9,...,3200,3201,3202,3203,3204,3205,3206,3207,3208,3209
0,101,1045,2134,1056,5987,5745,1999,1996,3698,2000,...,0,0,0,0,0,0,0,0,0,0
1,101,4914,2135,2007,1037,2516,2066,1996,2158,17420,...,0,0,0,0,0,0,0,0,0,0
2,101,3459,5696,12838,17399,21863,3367,9438,3870,5797,...,0,0,0,0,0,0,0,0,0,0
3,101,3459,2198,4190,10179,20722,2080,10799,3557,18314,...,0,0,0,0,0,0,0,0,0,0
4,101,4626,3958,13075,5420,6902,1047,2508,3804,9413,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1302,101,3735,3491,2012,1996,4407,2782,1997,2088,5988,...,0,0,0,0,0,0,0,0,0,0
1303,101,2142,2163,1057,1055,11343,6396,2474,3132,2770,...,0,0,0,0,0,0,0,0,0,0
1304,101,3058,2207,6396,2474,6974,2770,3091,6758,1054,...,0,0,0,0,0,0,0,0,0,0
1305,101,1045,2123,1056,2788,2079,4391,1997,3080,5691,...,0,0,0,0,0,0,0,0,0,0


In [133]:
y_for_ml = reviewers_data['ratings']
y_for_ml

0       0.05
1       0.05
2       0.05
3       0.05
4       0.09
        ... 
1302    0.96
1303    0.96
1304    0.97
1305    0.97
1306    0.99
Name: ratings, Length: 1307, dtype: float64

In [134]:
X_train, X_test, y_train, y_test = train_test_split(aligned_data_for_ML, y_for_ml, test_size=0.3, random_state=42)

In [135]:
linear_regression = LinearRegression()

In [136]:
linear_regression.fit(X_train, y_train)

LinearRegression()

In [137]:
predicted = linear_regression.predict(X_test)
predicted

array([ 0.224264  ,  0.36960753,  0.38058051, -0.33329035,  0.17131397,
       -0.30280495,  1.53883863,  1.29529425,  0.95226318,  1.51264079,
        0.37971692,  0.0948435 ,  1.1717501 ,  0.33808233, -0.16073149,
        1.0438038 ,  1.02718805, -0.50260003,  1.45031349,  0.38572847,
        0.66575964,  0.14690191,  0.10457454,  0.74615951,  1.01129483,
        0.14767621,  0.85625431,  0.36100391,  1.06730453,  0.10693876,
        1.4844989 ,  0.5645912 ,  1.11357481,  0.07254842,  1.43681138,
        1.06536184,  0.127397  ,  0.01282438,  0.73065286,  1.11636747,
        1.0662557 ,  1.23945832,  0.70883857,  0.65897705, -0.33224935,
        1.02990936,  0.19089122,  0.85824926,  1.5628014 ,  0.675493  ,
        0.19369669,  0.48451699,  0.74561168,  0.51942441,  0.74918064,
        1.24951559,  0.86314212,  0.20802421,  1.66710619,  1.38446633,
        1.46043062,  0.86991491, -0.20002401,  0.47990381,  0.95030733,
        0.28948297,  0.5028834 ,  1.06225089,  1.9722885 ,  0.54

In [138]:
metrics.mean_squared_error(y_test, predicted)

0.3505657965451288

In [139]:
metrics.mean_absolute_error(y_test, predicted)

0.4624526490890243